In [1]:
import sys
sys.path.append('/home/rneves/turbofan/')

import helper
from helper import *

%matplotlib inline


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

/home/rneves/anaconda3/envs/python3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/rneves/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datasets = []

path = '/datadrive/Turbofan_Engine/' #path to .txt files

text_files = [f for f in os.listdir(path) if f.endswith('.txt') and not f.startswith('r')]

dataframe = [os.path.splitext(f)[0] for f in text_files]

sensor_columns = ["sensor {}".format(s) for s in range(1,22)]

info_columns = ['dataset_id', 'unit_id','cycle','setting 1', 'setting 2', 'setting 3']

label_columns = ['dataset_id', 'unit_id', 'rul']

settings = ['setting 1', 'setting 2', 'setting 3']

test_data = []
train_data = []
RUL_data = []

for file in text_files:
    print(file)
    
    if re.match('RUL*', file):
        subset_df = pd.read_csv(path+file, delimiter=r"\s+", header=None)
        unit_id = range(1, subset_df.shape[0] + 1)
        subset_df.insert(0, 'unit_id', unit_id)
        dataset_id = basename(file).split("_")[1][:5]
        subset_df.insert(0, 'dataset_id', dataset_id)    
        RUL_data.append(subset_df)
    
    if re.match('test*', file):
        subset_df = pd.read_csv(path+file, delimiter=r"\s+", header=None, usecols=range(26))
        dataset_id = basename(file).split("_")[1][:5]
        subset_df.insert(0, 'dataset_id', dataset_id)  
        test_data.append(subset_df)
    
    if re.match('train*', file):
        subset_df = pd.read_csv(path+file, delimiter=r"\s+", header=None, usecols=range(26))  
        dataset_id = basename(file).split("_")[1][:5]
        subset_df.insert(0, 'dataset_id', dataset_id)   
        train_data.append(subset_df)


df_train = pd.concat(train_data, ignore_index=True)
df_train.columns = info_columns + sensor_columns               
df_train.sort_values(by=['dataset_id', 'unit_id', 'cycle'], inplace=True)

df_test = pd.concat(test_data, ignore_index=True)
df_test.columns = info_columns + sensor_columns
df_test.sort_values(by=['dataset_id', 'unit_id', 'cycle'], inplace=True)

df_RUL = pd.concat(RUL_data, ignore_index=True)
df_RUL.columns = label_columns
df_RUL.sort_values(by=['dataset_id', 'unit_id'], inplace=True)

RUL_train = df_train.groupby(['dataset_id', 'unit_id'])['cycle'].max()
RUL_test = df_test.groupby(['dataset_id', 'unit_id'])['cycle'].max() + df_RUL.groupby(['dataset_id', 'unit_id'])['rul'].max()

df_train['RUL'] = df_train.apply(lambda r: get_RUL(r, RUL_train), axis=1)
df_test['RUL'] = df_test.apply(lambda r: get_RUL(r, RUL_test), axis=1)

test_FD003.txt
train_FD002.txt
test_FD004.txt
RUL_FD003.txt
test_FD001.txt
train_FD001.txt
RUL_FD001.txt
RUL_FD002.txt
train_FD003.txt
train_FD004.txt
test_FD002.txt
RUL_FD004.txt


In [3]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=3000, prediction_data=True).fit(df_train[['setting 1', 'setting 2', 'setting 3']])

train_labels, strengths = hdbscan.approximate_predict(clusterer, df_train[['setting 1', 'setting 2', 'setting 3']])
test_labels, strengths = hdbscan.approximate_predict(clusterer, df_test[['setting 1', 'setting 2', 'setting 3']])

df_train['HDBScan'] = train_labels
df_test['HDBScan'] = test_labels

In [5]:
df_train.set_index(['dataset_id', 'unit_id'], inplace=True)
df_test.set_index(['dataset_id', 'unit_id'], inplace=True)

In [8]:
pd.to_pickle(df_train, '/datadrive/Turbofan_Engine/df_train_cluster.pkl')
pd.to_pickle(df_test, '/datadrive/Turbofan_Engine/df_test_cluster.pkl')

In [11]:
idx_train = df_train.index.to_series().unique()[np.arange(len(df_train.index.to_series().unique()))][[0,1,2,3,4,100,101,102,103,104,360,361,362,363,364, 460,461,462,463,464]]
idx_test = df_test.index.to_series().unique()[np.arange(len(df_test.index.to_series().unique()))][[0,1,2,3,4,100,101,102,103,104,359,360,361,362,363, 459,460,461,462,463]]

train = df_train.loc[idx_train]
test = df_test.loc[idx_test]

pd.to_pickle(train, '/datadrive/Turbofan_Engine/df_train_sample_cluster.pkl')
pd.to_pickle(test, '/datadrive/Turbofan_Engine/df_test_sample_cluster.pkl')